In [11]:
import numpy as np
from collections import namedtuple
from scipy.optimize import minimize_scalar

In [12]:
class MinAlgorithm:
    
    def __init__(self, f, grad=None, hess=None):
        self.func = f
        self.grad = grad
        self.hess = hess
        self.comp_grad = None
        self.comp_hess = None
        
    def minimize(self, starting_point, params):
        raise NotImplementedError()
        
default_params = ["MaxNumIter", "tolGrad", "tolIter", "gradHess", "alpha", "beta", "theta"]
p = dict(zip(default_params, len(default_params) * [None]))
p["MaxNumIter"] = 1000
p["tolGrad"] = 10e-5
p["tolIter"] = 10e-5

In [24]:
class SteepestDescent(MinAlgorithm):
    
    def set_line_search(self, name):
        if name == "fminsearch":
            self.line_search = lambda f: minimize_scalar(f, method="brent")
        elif name == "paso_limitado":
            self.line_search = None
        elif name == "orden_cero":
            self.line_search = None
        elif name == "armijo":
            self.line_search = None
        else:
            raise Exception("El metodo de busqueda {} no existe".format(name))
            
    def line_minimize(self, g):
        res = self.line_search(g)
        return res.x
        
    def minimize(self, starting_point, params):
        assert getattr(self, "line_search", None)
        point, old_point = starting_point, starting_point + 1
        steps = 0
        d = -self.grad(point)
        result = 100
        while steps < params["MaxNumIter"] and result > params["tolGrad"]\
                    and np.linalg.norm(point - old_point) > params["tolIter"]:
            steps += 1
            phi = lambda alpha: self.func((point + alpha * d).flatten())
            min_alpha = self.line_minimize(phi)
            old_point = point
            point += min_alpha * d
            d = -self.grad(point)
            result = np.linalg.norm(d)
        return point

In [25]:
f = lambda v: v[0] ** 2 + v[1] ** 2
grad_f = lambda v: np.array([2 * v[0], 2 * v[1]])
minimizer_alg = SteepestDescent(f, grad_f)
minimizer_alg.set_line_search("fminsearch")
print minimizer_alg.minimize(np.array([2, 3]), p)

[0 0]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.


In [26]:
f = lambda v: np.sum(np.array([(v[0] + 2 * v[1] - 7) ** 2, (2 * v[0] + v[1] - 5) ** 2]))
grad_f = lambda v: np.array([2 * (v[0] + 2 * v[1] - 7) + 4 * (2 * v[0] + v[1] - 5), 4 * (v[0] + 2 * v[1] - 7) + 2 * (2 * v[0] + v[1] - 5)])
minimizer_alg = SteepestDescent(f, grad_f)
minimizer_alg.set_line_search("fminsearch")
print minimizer_alg.minimize(np.array([2, 3]), p)

[1 2]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
